<a href="https://colab.research.google.com/github/Puisque/object-natural-scenes-detection/blob/main/Step_0_Dataset_analysis_%26_balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

import torchvision
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **Overview**
Step:
1. Download a csv file that contains information of each image **to local machine**
2. Upload the csv file to /content/gdrive/MyDrive/Capstone/Code 
3. Analyze the dataset based on the csv file

## **Preparation**

Download a csv file that contains information of each image to local machine



In [ ]:
image_path = '/content/gdrive/MyDrive/Capstone/Code/natural scene dataset'
file_name_total = 'Nature Scene Dataset Analysis Result.csv'

In [ ]:
#Load images 
nature_scene__dataset = torchvision.datasets.ImageFolder(root=image_path)
print(len(nature_scene__dataset.samples))

FileNotFoundError: ignored

In [ ]:
def download_data_to_csv(data_list,file_name):
  name_list = np.asarray([each[0] for each in data_list.samples])
  image_name = []
  class_name =[]
  folder = []
  for eachPath in name_list:
    image_name.append(eachPath.split('/')[-1])
    class_name.append(eachPath.split('/')[-2])
    folder.append(eachPath.split('/')[-3]) 

  #format
  file_format =  pd.DataFrame({'name': image_name[:],'className':class_name[:],'folder':folder[:]})
  file_format.astype({"name": str,'className': str,'folder':str})
  
  #download data to csv file
  file_format.to_csv(file_name) 
  files.download(file_name)

In [ ]:
download_data_to_csv(nature_scene__dataset,file_name_total)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Analysis - Before Balance**

Analyze the nature scene dataset based on the csv file

In [ ]:
header = ['name','className','folder']

In [ ]:
def data_analysis(fileName):
  file = pd.read_csv(
    "/content/gdrive/MyDrive/Capstone/Code/"+fileName,
    usecols=header)
  
  table = file.groupby([file.folder,file.className])['name'].count() #group images by train/validation/test and its class
  
  return table

In [ ]:
data_table = data_analysis(file_name_total)

In [ ]:
data_table

folder      className
test        buildings     442
            forest        442
            glacier       442
            mountain      442
            sea           442
            street        442
train       buildings    1749
            forest       1829
            glacier      1962
            mountain     2070
            sea          1832
            street       1940
validation  buildings     437
            forest        474
            glacier       553
            mountain      525
            sea           510
            street        501
Name: name, dtype: int64

## **Balance Dataset - One-time movement**

Suggested approach: 
1. Perform file movement in **one folder**(train/validation/test) at one time
2. If we want to make all classes have the same number of samples, the **`num_samples_{}`** hould be equal to the number of samples of the smallest class

Note: **Before careful in running the below cells**, or it might leads to duplicate movement

In [ ]:
import shutil 
import random

In [ ]:
storage_path = '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/'

In [ ]:
#Train set

train_dict = data_table['train'].to_dict()
print(train_dict)
num_samples_train = min(train_dict.values())
print('min = ',num_samples_train)

{'buildings': 1749, 'forest': 1829, 'glacier': 1962, 'mountain': 2070, 'sea': 1832, 'street': 1940}
min =  1749


In [ ]:
#Validation set

val_dict = data_table['validation'].to_dict()
print(val_dict)
num_samples_val = min(val_dict.values())
print('min = ',num_samples_val)

{'buildings': 437, 'forest': 474, 'glacier': 553, 'mountain': 525, 'sea': 510, 'street': 501}
min =  437


In [ ]:
nature_scene__dataset_train = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/Capstone/Code/natural scene dataset /natural scene/train')

In [ ]:
nature_scene__dataset_val = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/Capstone/Code/natural scene dataset /natural scene/validation')

In [ ]:
#lists of image(file) name
train_nameList = nature_scene__dataset_train.samples
val_nameList = nature_scene__dataset_val.samples

### Balance the **train** folder

In [ ]:
for class_name, count in train_dict.items():
  print('Moving image in [ {} ] folder, current count = {}'.format(class_name,count))

  move_count = count -  num_samples_train

  print('Plan to move {} images'.format(move_count))

  while move_count !=0:
    selectedFile = random.choice(train_nameList) # randomly choose which image to move out
    folder_name = selectedFile[0].split('/')[-2]
    if folder_name == class_name:
      try:
        shutil.move(selectedFile[0],storage_path + folder_name)
        move_count -=1
      except Exception as e:
          print(e)
  
  print('Complete image movement in [ {} ] folder.'.format(class_name))
  print('---------------------------------------')


Moving image in [ buildings ] folder, current count = 1749
Plan to move 0 images
Complete image movement in [ buildings ] folder.
---------------------------------------
Moving image in [ forest ] folder, current count = 1829
Plan to move 80 images
Complete image movement in [ forest ] folder.
---------------------------------------
Moving image in [ glacier ] folder, current count = 1962
Plan to move 213 images
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/4952.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/4800.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/5330.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/5059.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/4402.png' already exists
Destination path '/content/gdrive/MyDri

### **Check Result - Train Set**

In [ ]:
after_nature_scene__dataset_train = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/Capstone/Code/natural scene dataset /natural scene/train')

In [ ]:
file_name_train_after = 'After Balance - Train.csv'

In [ ]:
download_data_to_csv(after_nature_scene__dataset_train,file_name_train_after)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data_table_train_balance = data_analysis(file_name_train_after)
data_table_train_balance

folder  className
train   buildings    1749
        forest       1749
        glacier      1749
        mountain     1749
        sea          1749
        street       1749
Name: name, dtype: int64

### Balance the **validation** folder

In [ ]:
for class_name, count in val_dict.items():
  print('Moving image in [ {} ] folder, current count = {}'.format(class_name,count))

  move_count = count -  num_samples_val

  print('Plan to move {} images'.format(move_count))

  while move_count !=0:
    selectedFile = random.choice(val_nameList) # randomly choose which image to move out
    folder_name = selectedFile[0].split('/')[-2]
    if folder_name == class_name:
      try:
        shutil.move(selectedFile[0],storage_path + folder_name)
        move_count -=1
      except Exception as e:
          print(e)
  
  print('Complete image movement in [ {} ] folder.'.format(class_name))
  print('---------------------------------------')


Moving image in [ buildings ] folder, current count = 437
Plan to move 0 images
Complete image movement in [ buildings ] folder.
---------------------------------------
Moving image in [ forest ] folder, current count = 474
Plan to move 37 images
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/forest/641.png' already exists
Complete image movement in [ forest ] folder.
---------------------------------------
Moving image in [ glacier ] folder, current count = 553
Plan to move 116 images
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/1419.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/1025.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/1139.png' already exists
Destination path '/content/gdrive/MyDrive/Capstone/Code/dataset storage bin/glacier/1310.png' already exists
Destination path '/content/gdrive/MyDrive/Ca

### **Check Result - Validation Set**

In [ ]:
after_nature_scene__dataset_validation = torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/Capstone/Code/natural scene dataset /natural scene/validation')

In [ ]:
file_name_val_after = 'After Balance - Validation.csv'

In [ ]:
download_data_to_csv(after_nature_scene__dataset_validation,file_name_val_after)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data_table_val_balance = data_analysis(file_name_val_after)
data_table_val_balance

folder      className
validation  buildings    437
            forest       437
            glacier      437
            mountain     437
            sea          437
            street       437
Name: name, dtype: int64

## **Analysis - After Balance**

Analyze the nature scene dataset based on the csv file

In [ ]:
#Load images
nature_scene__dataset_after = torchvision.datasets.ImageFolder(root=image_path)
print(len(nature_scene__dataset_after.samples))

15768


In [ ]:
file_name_total_after = 'After Balance - Nature Scene Dataset Analysis Result.csv'

In [ ]:
download_data_to_csv(nature_scene__dataset_after,file_name_total_after)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data_table_after = data_analysis(file_name_total_after)
data_table_after

folder      className
test        buildings     442
            forest        442
            glacier       442
            mountain      442
            sea           442
            street        442
train       buildings    1749
            forest       1749
            glacier      1749
            mountain     1749
            sea          1749
            street       1749
validation  buildings     437
            forest        437
            glacier       437
            mountain      437
            sea           437
            street        437
Name: name, dtype: int64